In [1]:
import numpy as np
import pandas as pd
from optbinning import OptimalBinning

/home/huangdan/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/huangdan/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


(CVXPY) Feb 29 03:16:45 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Feb 29 03:16:45 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.8.3296). Expected < 9.8.0. Please open a feature request on cvxpy to enable support for this version.')


### Load data and repeat the preprocess.

In [2]:
file_path = "/hdd1/sams/census/census.arff"
with open(file_path, 'r') as f:
    lines = f.readlines()

In [3]:
flag = False
rows = []
for line in lines:
    if line.strip() == "@data":
        flag = True
        continue
    
    if not flag:
        continue
    
    row = [i for i in line.strip().split(',')]
    if len(row) != 42:
        print(row)
        break
    rows.append(row) 
del lines

In [4]:
df = pd.DataFrame(rows)

In [5]:
numerical_cols = [0,2,3,5,16,17,18,24,30,36,38,39,40]
len(numerical_cols)

13

In [6]:
bin_cols = []
print(f"#col: {len(df.columns)}")
for col in numerical_cols:
    n = len(df[col].unique())
    if n > 7:
        print(f"{col}:{n}")
        bin_cols.append(col)

#col: 42
0:91
2:52
3:47
5:1425
16:133
17:114
18:1675
24:123232
39:53


In [7]:
# transfer these bin cols to float
for col in bin_cols:
    try:
        df[col] = df[col].astype(float)
    except:
        print(col)

In [8]:
# label to 1,0
map_y = {"' - 50000.'": 0, "' 50000+.'":1}
df[41] = df[41].map(map_y)

In [9]:
Y = df[41]
del df[41]
# X = df[range(69)]
X = df

In [10]:
def optbinning_data(df:pd.DataFrame, column:int, y:np.array):
    x = df[column].values

    optb = OptimalBinning(name="23", dtype="numerical", solver="cp",
                      monotonic_trend="auto", 
                      max_n_prebins=100,
                      min_prebin_size=0.001, time_limit=200)
    optb.fit(x,y)
    res = optb.transform(x, metric="indices")
    bin_number = max(res)
    status = optb.status
    set_n = len(df[column].unique())
    print(f"bin column:{column}, set size {set_n}, bin status {status}, bin_number {bin_number}")
    return res

In [11]:
for col in bin_cols:
    v = optbinning_data(X, col, Y.values)
    X[col] = v 

bin column:0, set size 91, bin status OPTIMAL, bin_number 45
bin column:2, set size 52, bin status OPTIMAL, bin_number 5
bin column:3, set size 47, bin status OPTIMAL, bin_number 1
bin column:5, set size 1425, bin status OPTIMAL, bin_number 5
bin column:16, set size 133, bin status OPTIMAL, bin_number 6
bin column:17, set size 114, bin status OPTIMAL, bin_number 6
bin column:18, set size 1675, bin status OPTIMAL, bin_number 16
bin column:24, set size 123232, bin status FEASIBLE, bin_number 11
bin column:39, set size 53, bin status OPTIMAL, bin_number 17


In [12]:
# tokenization
col_to_value2token = {}
cnt = 0
for col in X.columns:
    col_values = X[col].values
    col_to_value2token[col] = {}
    col_values_set = set(col_values)
    for val in col_values_set:
        if val not in col_to_value2token[col]:
            col_to_value2token[col][val] = cnt
            cnt += 1
print(cnt)

532


In [18]:
predicate_bitmap = [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
sql = [532, 533, 534, 535, 536, 537, 538, 539, 540, 133, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 281, 555, 556, 557, 558, 559, 560, 561, 356, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572]

In [22]:
def parse_sql(predicate_bitmap, sql):
    for col_idx, i in enumerate(predicate_bitmap):
        if i == 0:
            continue
        cname = "V"+ str(col_idx + 1)
        print(f"{cname} Column")
        col_name = X.columns[col_idx]
        value2token = col_to_value2token[col_name]
        token2value = {v:k for k,v in value2token.items()}
        filter_token = sql[col_idx]
        filter_value = token2value[filter_token]
        print(f'Column {cname}, filter condition {filter_value}')

In [23]:
parse_sql(predicate_bitmap, sql)

V10 process
Column V10, filter condition ' Adm support including clerical'
V23 process
Column V23, filter condition ' Child <18 never marr RP of subfamily'
V31 process


KeyError: 356

In [24]:
col_to_value2token[X.columns[30]]

{'4': 357, '0': 358, '5': 359, '2': 360, '6': 361, '1': 362, '3': 363}

In [25]:
len(X.columns)

41